# Adding the Spotify Songs dataset to a database

This week, I'll be using the [Tidy Tuesday Spotify Songs dataset](https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-01-21/readme.md) to practice working with SQL a bit. If you're curious, the link to the dataset has more information on how this was gathered too. 

Let's start by getting the data and setting up. 

## 0. Data and setup

In [1]:
# Packages needed 
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

# Set option to display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# Read in spotify dataset from github url and take a peek
spotify_df = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv")
spotify_df.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.748,0.916,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.726,0.815,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.675,0.931,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.718,0.930,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.650,0.833,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


Let's also create a SQLite3 database on disk to use to store our data later. Note: if you don't have `sqlmagic`,  you may need to install it first as follows:

In [3]:
! python3 -m pip install --quiet ipython-sql

In [4]:
%load_ext sql

Then, you can make a new database (or connect to it, if it already exists): 

In [5]:
%sql sqlite:///data/spotify.db

## 1. Normalize data to Third Normal Form 

Before we populate our database, it's good to normalize it; this enables us to remove redundancies and prevent inconsistencies later. 


### 1.1 Normalize data into First Normal Form 

First, let's get the data into First Normal Form (1NF). This means we want the data table (at least for now, let's consider this to be `spotify_df`) to have: 

1. A primary key (a unique, non-null column identifying each row). 
2. No repeating groups of columns
3. Each cell contains a single value

#### 1.1.1 Identify or create primary key

Let's start by looking at the columns available: 

In [6]:
spotify_df.columns

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='object')

At first glance, it seems like `track_id` might be a good contender for a primary key, but let's confirm its values are unique and non-null for each row: 

In [7]:
# Check if length of unique track_id values is equal to number of rows 
spotify_df.shape[0] == len(spotify_df["track_id"].unique())

False

Hm, that doesn't seem to be the case. Let's look at one duplicated track_id to get a better sense for what a reasonable primary key could be: 

In [8]:
# Get first track_id that's duplicated
first_duplicated_track_id = (spotify_df.
                             loc[spotify_df.duplicated(subset=['track_id'])]["track_id"].
                             iloc[0])

# Inspect rows of duplicated track_id
spotify_df.loc[spotify_df["track_id"] == first_duplicated_track_id]

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
739,1HfMVBKM75vxSfsQ5VefZ5,Lose You To Love Me,Selena Gomez,93,3tBkjgxDqAwss76O1YHsSY,Lose You To Love Me,2019-10-23,Todo Éxitos,2ji5tRQVfnhaX1w9FhmSzk,pop,dance pop,0.505,0.34,4,-9.005,1,0.0438,0.576,0.0,0.21,0.0916,101.993,206459
1299,1HfMVBKM75vxSfsQ5VefZ5,Lose You To Love Me,Selena Gomez,93,3tBkjgxDqAwss76O1YHsSY,Lose You To Love Me,2019-10-23,Pop - Pop UK - 2019 - Canadian Pop - 2019 - Pop,46Cl6dmeiylK6TRGXr7hHe,pop,post-teen pop,0.505,0.34,4,-9.005,1,0.0438,0.576,0.0,0.21,0.0916,101.993,206459
18320,1HfMVBKM75vxSfsQ5VefZ5,Lose You To Love Me,Selena Gomez,93,3tBkjgxDqAwss76O1YHsSY,Lose You To Love Me,2019-10-23,2020 Hits & 2019 Hits – Top Global Tracks 🔥🔥🔥,4JkkvMpVl4lSioqQjeAL0q,latin,latin pop,0.505,0.34,4,-9.005,1,0.0438,0.576,0.0,0.21,0.0916,101.993,206459
19730,1HfMVBKM75vxSfsQ5VefZ5,Lose You To Love Me,Selena Gomez,93,3tBkjgxDqAwss76O1YHsSY,Lose You To Love Me,2019-10-23,2020 Hits & 2019 Hits – Top Global Tracks 🔥🔥🔥,4JkkvMpVl4lSioqQjeAL0q,latin,latin hip hop,0.505,0.34,4,-9.005,1,0.0438,0.576,0.0,0.21,0.0916,101.993,206459
21555,1HfMVBKM75vxSfsQ5VefZ5,Lose You To Love Me,Selena Gomez,93,3tBkjgxDqAwss76O1YHsSY,Lose You To Love Me,2019-10-23,Most Popular 2020 TOP 50,1fqkbjEACMlekdddm5aobE,r&b,urban contemporary,0.505,0.34,4,-9.005,1,0.0438,0.576,0.0,0.21,0.0916,101.993,206459
23641,1HfMVBKM75vxSfsQ5VefZ5,Lose You To Love Me,Selena Gomez,93,3tBkjgxDqAwss76O1YHsSY,Lose You To Love Me,2019-10-23,"Latest Hits 2020 - Pop, Hip Hop & RnB",7FqZlaYKkQmVnguJbHuj2a,r&b,hip pop,0.505,0.34,4,-9.005,1,0.0438,0.576,0.0,0.21,0.0916,101.993,206459
30388,1HfMVBKM75vxSfsQ5VefZ5,Lose You To Love Me,Selena Gomez,93,3tBkjgxDqAwss76O1YHsSY,Lose You To Love Me,2019-10-23,2010 - 2011 - 2012 - 2013 - 2014 - 2015 - 2016...,2DjIfVDXGYDgRxw7IJTKVb,edm,pop edm,0.505,0.34,4,-9.005,1,0.0438,0.576,0.0,0.21,0.0916,101.993,206459


Ok-- it seems like the `track_id` is redundant here because it's on multiple playlists. We could combine the `track_id` with `playlist_id` to create a unique ID per row, but considering that they have to do with different things it probably makes more sense to just split the data frame into **two** tables (track and playlist) and allocate relevant columns accordingly. So, let's do that: 

In [9]:
spotify_df.columns

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='object')

In [10]:
# columns relevant to track
track_columns = ['track_id', 
                 'track_name', 
                 'track_artist', 
                 'track_popularity',
                 'track_album_id', 
                 'track_album_name', 
                 'track_album_release_date',
                 'danceability', 
                 'energy', 
                 'key', 
                 'loudness', 
                 'mode', 
                 'speechiness',
                 'acousticness', 
                 'instrumentalness', 
                 'liveness', 
                 'valence', 
                 'tempo',
                 'duration_ms'] 

# columns relevant to playlist 
playlist_columns = ['playlist_id',
                    'playlist_name', 
                    'playlist_genre', 
                    'playlist_subgenre']

# Create a dictionary of 2 dataframes (track_df and playlist_df)
spotify_df_dict = {'track' : spotify_df.loc[:,track_columns].drop_duplicates(),
                  'playlist' : spotify_df.loc[:,playlist_columns].drop_duplicates()}

Now let's confirm that `track_id` and `playlist_id` are unique and non-null within each DataFrame: 

In [11]:
# Check track_id is unique per row in track df
print("TRACK:")
print("Is there a unique id per row?")
print(spotify_df_dict["track"].shape[0] == len(spotify_df_dict["track"]["track_id"].unique()))

# Check track_id has no NA values
print("Are there NA values?")
print(spotify_df_dict["track"]["track_id"].isnull().any())

# Check playlist_id is unique per row in playlist df
print("\nPLAYLIST:")
print("Is there a unique id per row?")
print(spotify_df_dict["playlist"].shape[0] == len(spotify_df_dict["playlist"]["playlist_id"].unique()))

# Check playlist_id has no NA values
print("Are there NA values?")
print(spotify_df_dict["playlist"]["playlist_id"].isnull().any())

TRACK:
Is there a unique id per row?
True
Are there NA values?
False

PLAYLIST:
Is there a unique id per row?
False
Are there NA values?
False


Seems like `playlist_id` still isn't unique. Let's look more closely at the rows where `playlist_id` is duplicated: 

In [12]:
duplicated_playlist_ids = (spotify_df_dict["playlist"].
 loc[spotify_df_dict["playlist"].duplicated(subset=['playlist_id'])]["playlist_id"])

spotify_df_dict["playlist"][spotify_df_dict["playlist"]["playlist_id"].
                            isin(duplicated_playlist_ids)].sort_values(by=["playlist_id"])

,playlist_id,playlist_name,playlist_genre,playlist_subgenre
29945,25ButZrVb1Zj1MJioMs09D,EDM 2020 House & Dance,edm,pop edm
27216,25ButZrVb1Zj1MJioMs09D,EDM 2020 House & Dance,edm,electro house
30804,2CJsD3fcYJWcliEKnwmovU,TOP 50 GLOBAL 2020 UPDATED WEEKLY 🌍🎶 WORLDWIDE,edm,pop edm
23436,2CJsD3fcYJWcliEKnwmovU,TOP 50 GLOBAL 2020 UPDATED WEEKLY 🌍🎶 WORLDWIDE,r&b,hip pop
1067,37i9dQZF1DWTHM4kX49UKs,Ultimate Indie Presents... Best Indie Tracks o...,pop,dance pop
22829,37i9dQZF1DWTHM4kX49UKs,Ultimate Indie Presents... Best Indie Tracks o...,r&b,hip pop
10387,37i9dQZF1DX4OjfOteYnH8,Flow Selecto,rap,trap
19687,37i9dQZF1DX4OjfOteYnH8,Flow Selecto,latin,reggaeton
12900,3Ho3iO0iJykgEQNbjB2sic,"Classic Rock 70s 80s 90s, Rock Classics - 70s ...",rock,classic rock
15155,3Ho3iO0iJykgEQNbjB2sic,"Classic Rock 70s 80s 90s, Rock Classics - 70s ...",rock,hard rock


The issue seems to be that some playlists are listed under two subgenres. We want to keep a single value per column so combining the redundant subgenres isn't a good idea. So, let's instead just create a new unique id for the playlist dataframe:

In [13]:
spotify_df_dict["playlist"]["playlist_uid"] = list(range(0, spotify_df_dict["playlist"].shape[0]))

In [14]:
# Check playlist_uid is unique per row in playlist df
print("Is there a unique id per row?")
print(spotify_df_dict["playlist"].shape[0] == len(spotify_df_dict["playlist"]["playlist_uid"].unique()))

# Check playlist_uid has no NA values
print("Are there NA values?")
print(spotify_df_dict["playlist"]["playlist_uid"].isnull().any())

Is there a unique id per row?
True
Are there NA values?
False


Great! Moving on. 

#### 1.1.2 Check that there are no repeating columns 

For the track DataFrame, none of the columns seem to repeat: 

In [15]:
spotify_df_dict["track"].head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,0.748,0.916,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,0.726,0.815,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,0.675,0.931,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,0.718,0.930,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,0.650,0.833,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


For the playlist DataFrame (since we added a unique identifying column), there are no repeating groups of columns:

In [16]:
spotify_df_dict["playlist"].head()

,playlist_id,playlist_name,playlist_genre,playlist_subgenre,playlist_uid
0,37i9dQZF1DXcZDD7cfEKhW,Pop Remix,pop,dance pop,0
70,37i9dQZF1DWZQaaqNMbbXa,Dance Pop,pop,dance pop,1
167,37i9dQZF1DX2ENAPP1Tyed,Dance Room,pop,dance pop,2
223,37i9dQZF1DWSJHnPb1f0X3,Cardio,pop,dance pop,3
272,37i9dQZF1DX6pH08wMhkaI,Dance Pop Hits,pop,dance pop,4


#### 1.1.3 Each cell contains a single value

From our inspection of the DataFrames in 1.2, we see that each cell contains a single value; so, this requirement is satisfied.  

### 2. Normalize data into Second Normal Form 

Now we're ready to get our data into Second Normal form ("2NF"). This means we want each table to have all columns in each row to depend fully on candidate keys. Basically, we'll ask if each column in the two tables we have so far serve to describe what the primary key identifies. 

#### 2.1 Playlist table 

The columns of the playlist table are as follows: 

In [17]:
spotify_df_dict["playlist"].columns

Index(['playlist_id', 'playlist_name', 'playlist_genre', 'playlist_subgenre',
       'playlist_uid'],
      dtype='object')

Each column describes what the primary key identifies (a playlist), so 2NF is satisfied for this table. 

#### 2.2. Track table

The columns of the track table are as follows: 

In [18]:
spotify_df_dict["track"].columns

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='object')

Here, we notice that some of the columns (specifically, `track_album_id`, `track_album_name`, `track_album_release_date`) relate to the **album** of the track, not the track itself (as the other columns do). So, let's move the album-related columns to their own table: 

In [19]:
album_columns = ["track_album_id",
                "track_album_name",
                "track_album_release_date"]

# Create new DataFrame of album information, dropping duplicates
spotify_df_dict["album"] = spotify_df_dict["track"][album_columns].drop_duplicates()

# Check that track_album_id is an appropriate primary key
print("Is there a unique id per row?")
print(spotify_df_dict["album"].shape[0] == len(spotify_df_dict["album"]["track_album_id"].unique()))

# Check playlist_uid has no NA values
print("Are there NA values?")
print(spotify_df_dict["album"]["track_album_id"].isnull().any())

Is there a unique id per row?
True
Are there NA values?
False


Great! Now let's remove those columns from the track DataFrame and we should be all set with 2NF: 

In [20]:
spotify_df_dict["track"].drop(album_columns, axis=1, inplace=True)

# Confirm that album columns were removed
spotify_df_dict["track"].columns

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='object')

### 3. Normalize data into Third Normal Form (3NF)

Lastly, we want to confirm that there are no transitive dependencies between non-candidate columns. 

#### 3.1 Album table

Let's look at the columns again: 

In [21]:
spotify_df_dict["album"].head()

,track_album_id,track_album_name,track_album_release_date
0,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14
1,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13
2,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05
3,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19
4,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05


Each column depends on the album_id (or row number), so there are no 3NF violations. 

#### 3.2 Playlist table 

In [22]:
spotify_df_dict["playlist"].head()

,playlist_id,playlist_name,playlist_genre,playlist_subgenre,playlist_uid
0,37i9dQZF1DXcZDD7cfEKhW,Pop Remix,pop,dance pop,0
70,37i9dQZF1DWZQaaqNMbbXa,Dance Pop,pop,dance pop,1
167,37i9dQZF1DX2ENAPP1Tyed,Dance Room,pop,dance pop,2
223,37i9dQZF1DWSJHnPb1f0X3,Cardio,pop,dance pop,3
272,37i9dQZF1DX6pH08wMhkaI,Dance Pop Hits,pop,dance pop,4


Here, `playlist_subgenre` violates 3NF, because it only depends on the `playlist_uid` via the `playlist_genre`. So, let's split that out (with `playlist_genre`) into a separate genre table: 

In [23]:
genre_columns = ["playlist_genre",
                "playlist_subgenre"]

# Create new DataFrame of genre information, dropping duplicates
spotify_df_dict["genre"] = spotify_df_dict["playlist"][genre_columns].drop_duplicates()

# Check that playlist_subgenre is an appropriate primary key
print("Is there a unique id per row?")
print(spotify_df_dict["genre"].shape[0] == len(spotify_df_dict["genre"]["playlist_subgenre"].unique()))

# Check playlist_uid has no NA values
print("Are there NA values?")
print(spotify_df_dict["genre"]["playlist_subgenre"].isnull().any())

Is there a unique id per row?
True
Are there NA values?
False


Great. Lastly, let's remove `playlist_genre` and `playlist_subgenre` from the playlist table: 

In [24]:
spotify_df_dict["playlist"].drop(["playlist_genre","playlist_subgenre"], axis=1, inplace=True)

# Confirm that album columns were removed
spotify_df_dict["playlist"].columns

Index(['playlist_id', 'playlist_name', 'playlist_uid'], dtype='object')

#### 3.3 Track table

In [25]:
spotify_df_dict["track"].head()

,track_id,track_name,track_artist,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,0.748,0.916,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,0.726,0.815,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,0.675,0.931,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,0.718,0.930,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,0.650,0.833,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


Each column depends on the track_id (or row number), so there are no 3NF violations. 

### 3. Populate tables of SQLite3 database

Now we're (almost) ready to populate our database! But first, we need to resolve an issue-- our tables are no longer linked! We need to add **foreign keys** (or, a field/column linking a given table to the primary key of a second table) because this enable us to maintain the relationships between the tables. 

#### 3.1 Add foreign keys to DataFrames 

We start by using the original DataFrame to make a map between each of the primary keys: `track_id`, `playlist_id` (* we will map to `playlist_uid` in a second), `playlist_subgenre`, and `track_album_id`):

In [26]:
original_id_columns = ["track_id",
                      "playlist_id",
                      "playlist_subgenre",
                      "track_album_id"]
id_map_df = spotify_df[original_id_columns].drop_duplicates()
id_map_df.head()

,track_id,playlist_id,playlist_subgenre,track_album_id
0,6f807x0ima9a1j3VPbc7VN,37i9dQZF1DXcZDD7cfEKhW,dance pop,2oCs0DGTsRO98Gh5ZSl2Cx
1,0r7CVbZTWZgbTCYdfa2P31,37i9dQZF1DXcZDD7cfEKhW,dance pop,63rPSO264uRjW1X5E6cWv6
2,1z1Hg7Vb0AhHDiEmnDE79l,37i9dQZF1DXcZDD7cfEKhW,dance pop,1HoSmj2eLcsrR0vE9gThr4
3,75FpbthrwQmzHlBJLuGdC7,37i9dQZF1DXcZDD7cfEKhW,dance pop,1nqYsOef1yKKuGOVchbsk6
4,1e8PAfcKUYoKkxPhrHqw4x,37i9dQZF1DXcZDD7cfEKhW,dance pop,7m7vv9wlQ4i0LFuJiE2zsQ


Now we add `playlist_uid` to make this a complete map: 

In [27]:
id_map_df = id_map_df.merge(spotify_df_dict["playlist"].loc[:,["playlist_id", "playlist_uid"]],
                           how="left",
                           on="playlist_id")
id_map_df.head()

,track_id,playlist_id,playlist_subgenre,track_album_id,playlist_uid
0,6f807x0ima9a1j3VPbc7VN,37i9dQZF1DXcZDD7cfEKhW,dance pop,2oCs0DGTsRO98Gh5ZSl2Cx,0
1,0r7CVbZTWZgbTCYdfa2P31,37i9dQZF1DXcZDD7cfEKhW,dance pop,63rPSO264uRjW1X5E6cWv6,0
2,1z1Hg7Vb0AhHDiEmnDE79l,37i9dQZF1DXcZDD7cfEKhW,dance pop,1HoSmj2eLcsrR0vE9gThr4,0
3,75FpbthrwQmzHlBJLuGdC7,37i9dQZF1DXcZDD7cfEKhW,dance pop,1nqYsOef1yKKuGOVchbsk6,0
4,1e8PAfcKUYoKkxPhrHqw4x,37i9dQZF1DXcZDD7cfEKhW,dance pop,7m7vv9wlQ4i0LFuJiE2zsQ,0


And now we add the relevant foreign keys to preserve the DataFrame's original links via joins:  

In [28]:
# Add participant_uid and album_id foreign keys to track df 
spotify_df_dict["track"] = (spotify_df_dict["track"].
                            merge(id_map_df.loc[:,["track_id","playlist_uid", "track_album_id"]],
                                 how="left",
                                 on="track_id"))

# Add playlist_subgenre foreign keys to playlist df
spotify_df_dict["playlist"] = (spotify_df_dict["playlist"].
                            merge(id_map_df.loc[:,["playlist_uid", "playlist_subgenre"]],
                                 how="left",
                                 on="playlist_uid"))

Ok, now we can finally...

#### 3.2 Populate tables of database

First, let's create an engine to the database we previously defined:

In [29]:
engine = create_engine('sqlite:///data/spotify.db')

And then we write each value of our dictionary of DataFrames to a table (pandas makes this really easy):

In [30]:
for key in spotify_df_dict.keys():
    spotify_df_dict[key].to_sql(key, con=engine)

### 4. Sample SQL query: Find the names of all playlists that contain instrumentals

Lastly, let's perform a sample query on our database. We see from [the dataset documentation](https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-01-21/readme.md) that the `instrumentalness` column (which is in the `track` table) assigns a value between 0 and 1 to each track, where values above 0.5 indicate the track has likely instrumental content. 

So, we want to: 

1. Identify which distinct `playlist_uid`s have `instrumentalness` > 0.5 in the track table.  
2. See which `playlist_name`s these `playlist_uids` belong to. 

We'll do this with a sub-query, as follows: 

In [31]:
%%sql 

SELECT DISTINCT playlist_name FROM playlist
WHERE playlist_uid IN (
    SELECT playlist_uid FROM track
    WHERE instrumentalness > 0.5
)

 * sqlite:///data/spotify.db
Done.


playlist_name
Pop Remix
Dance Room
Pop Warmup 130 BPM
Dance Pop
Dance Pop Tunes
Pop / Dance
Todo Éxitos
90s Dance Hits
Christian Dance Party
Pop Dance Hits


Ta-da! 